In [1]:
# This is AWP + exp67 seed=2022, maxlen=512, PL + decreasing max len + last 8 hidden layers
# Psuedo labels for this code is here - https://www.kaggle.com/code/rashmibanthia/fb3-psuedo-labels-1-from-exp47b/notebook?scriptVersionId=107695932 
# these were generated from exp47.

# This has 5 fold split of fb1 dataset - https://www.kaggle.com/code/rashmibanthia/fb3-fb1-merged

colab=False
vastai=False
local = True

if local: 
  data_dir =  "/home/rashmi/Documents/kaggle/feedback3/"
  
if colab:
  from google.colab import drive
  drive.mount('/content/gdrive')
  data_dir = 'gdrive/MyDrive/kaggle/2022/feedback3'

  !mkdir -p feedback1
  !cp gdrive/MyDrive/kaggle/2022/feedback_prize/data/train.zip feedback1
  !unzip -oqq gdrive/MyDrive/kaggle/2022/feedback_prize/data/train.zip -d feedback1/

  FEEDBACK1_TRAIN_CSV = 'gdrive/MyDrive/kaggle/2022/feedback_prize/data/train.csv'
  FEEDBACK1_TRAIN_DIR = 'feedback1/train/'

  !pip install wandb > /dev/null
  !pip install tokenizers > /dev/null
  !pip install transformers > /dev/null
  !pip install sentencepiece > /dev/null
  !pip install tez > /dev/null
  !pip install datasets > /dev/null
    
if vastai:
    data_dir = '/workspace/feedback3/'

In [2]:
# ====================================================
# Directory settings
# ====================================================
import os
EXP_NAME = 'exp117a_PL_9'
OUTPUT_DIR = f'{data_dir}/src/models_' + EXP_NAME + "/"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [3]:
import os, gc, pickle, math, time, random, copy, json
from glob import glob
import numpy as np
import pandas as pd
from sklearn import metrics
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed

from sklearn.model_selection import GroupKFold, StratifiedGroupKFold
from sklearn.metrics import log_loss

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from datasets import Dataset

# Mixed precision in Pytorch
from torch.cuda.amp import autocast, GradScaler

# For SWA
from torch.optim.swa_utils import AveragedModel, SWALR

from transformers import AutoConfig, AutoTokenizer, AutoModel
from transformers import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup, AdamW

import warnings
warnings.filterwarnings('ignore')

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

In [4]:
import transformers
transformers.__version__

'4.23.1'

In [5]:
class Config:
    # General settings
    competition_name = 'FeedbackPrize3'
    seed = 2022 #42
    debug = False
    train = True
    n_fold = 5
    print_freq = 100
    wandb = True
    val_strategy = "batch"
    val_steps = 250 #300
    # For model
    # model='gdrive/MyDrive/kaggle/2022/feedback2/pretrain/models_pretrain_debertav3large-expA/deberta-v3-large'
    # tokenizer_path='gdrive/MyDrive/kaggle/2022/feedback2/pretrain/models_pretrain_debertav3large-expA/deberta-v3-large/tokenizer'
    model = "microsoft/deberta-v3-large"
    tokenizer_path = "microsoft/deberta-v3-large"
    tokenizer = AutoTokenizer.from_pretrained(model)
    config = AutoConfig.from_pretrained(model)
    config.output_hidden_states = True
    config.hidden_dropout_prob = 0.
    config.attention_probs_dropout_prob = 0.
    
    scheduler='cosine'
    trn_fold = [0,1,2,3,4]
    max_len = 512
    batch_size = 4
    num_workers = os.cpu_count()
    # For training
    apex = True
    gradient_checkpointing = True
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    epochs = 4
    gradient_accumulation_steps = 1.
    max_grad_norm = 1000
    label_smoothing = 0.03
    num_labels=6
    # Optimizer
    lr = 1e-5
    weight_decay = 1e-2
    encoder_lr = 1e-5
    decoder_lr = 1e-5
    min_lr = 1e-6
    eps = 1e-6
    betas = (0.9, 0.999)
    # Scheduler
    scheduler_type = 'cosine'    # 'linear', 'cosine'
    if scheduler_type == 'cosine':
        num_cycles = 0.5
    num_warmup_steps = 100
    batch_scheduler = True
    gpu_optimize_config_adam = False
    TRAIN_FOLDS = f'{data_dir}/input/train_folds.csv'
    # For AWP
    use_awp = True
    if use_awp:
        start_awp_epoch = 1 
        awp_score_check = 0.49
        adv_lr = 2e-5
        adv_eps = 1e-3 #1e-2
#         adv_step = 1
    else:
        start_awp_epoch = epochs + 1

CFG = Config()
CFG.tokenizer.add_special_tokens({'additional_special_tokens': ['[PARAGRAPH]']})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
true_cols = ['cohesion', 'syntax', 'vocabulary','phraseology', 'grammar', 'conventions']
pred_cols = ['pred_cohesion', 'pred_syntax', 'pred_vocabulary','pred_phraseology', 'pred_grammar', 'pred_conventions']

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    
    import wandb

    try:
        # from kaggle_secrets import UserSecretsClient
        # user_secrets = UserSecretsClient()
        # secret_value_0 = user_secrets.get_secret("wandb_api")
        # wandb.login(key=secret_value_0)
        wandb.login(key='yourkeyhere')
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project=CFG.competition_name, 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type=EXP_NAME, #"train",
                     anonymous=anony)

wandb: Currently logged in as: rashmi. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/rashmi/.netrc


In [7]:
"""# Random seed"""

def seed_everything(seed, use_cuda = True):
    np.random.seed(seed) # cpu vars
    torch.manual_seed(seed) # cpu  vars
    random.seed(seed) # Python
    os.environ['PYTHONHASHSEED'] = str(seed) # Python hash building
    if use_cuda:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

seed_everything(CFG.seed)


In [8]:
all_pldata = pd.read_csv('/home/rashmi/Documents/kaggle/feedback3/input/fb3_fb1_merged/all_pldata.csv')
all_pldata.columns = ['text_id','full_text','token_length','kfold']
all_pldata

,text_id,full_text,token_length,kfold
0,423A1CA112E2,Phones Modern humans today are always on their...,442,0
1,A8445CABFECE,Phones & Driving Drivers should not be able to...,242,0
2,6B4F7A0165B9,Cell Phone Operation While Driving The ability...,364,0
3,E05C7F5C1156,People are debating whether if drivers should ...,671,0
4,50B3435E475B,Texting and driving Over half of drivers in to...,420,0
...,...,...,...,...
15137,0814426B27DF,Most people ask more than one person for advic...,512,4
15138,8F4B595CF9E7,Do you ever want more opinions and options whe...,639,4
15139,6B5809C83978,Has anyone ever gave you advice? Was the advic...,411,4
15140,AFEC37C2D43F,There has been at least one point in everyone'...,647,4


In [9]:
df_folds= pd.read_csv(CFG.TRAIN_FOLDS)
# df_folds['full_text'] = df_folds.full_text.apply(lambda x : resolve_encodings_and_normalize(x))
print(df_folds.shape)
df_folds.head(2)

(3911, 9)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,kfold
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,1
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,0


In [10]:
df_folds['full_text'] = df_folds.full_text.apply(lambda x: x.replace("\n\n"," [PARAGRAPH] "))

In [11]:
df_folds.kfold.value_counts(), df_folds[df_folds.kfold!=0].shape

(1    783
 0    782
 4    782
 3    782
 2    782
 Name: kfold, dtype: int64,
 (3129, 9))

In [12]:
def _prepare_training_data_helper(df, is_train):
      # print(df['full_text'])
      tok = CFG.tokenizer.encode(df['full_text'].values[0], add_special_tokens=True, max_length=CFG.max_len)
      label = df[true_cols].values[0]

      return {'input_ids': tok, 'label': label, 'essay_id': df['text_id'].values[0] }



def prepare_training_data(df, tokenizer, num_jobs, is_train):
    
    results = Parallel()(
        delayed(_prepare_training_data_helper)(gdf, is_train) for gn, gdf in tqdm(df.groupby('text_id'))
    )
    
    return results


In [13]:
folds = df_folds.copy()
fold=0
train_df = folds[folds["kfold"] != fold].reset_index(drop=True)#.head(500)
valid_df = folds[folds["kfold"] == fold].reset_index(drop=True)
NUM_JOBS = os.cpu_count()
training_samples_ = prepare_training_data(train_df, CFG.tokenizer, num_jobs=NUM_JOBS, is_train=True)

100%|██████████| 3129/3129 [00:02<00:00, 1302.85it/s]


In [14]:
class FeedbackDataset:
    def __init__(self, samples):
        self.samples = samples
        self.tokenizer = CFG.tokenizer

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        ids = self.samples[idx]["input_ids"]
        label = self.samples[idx]["label"]
        essay_id = self.samples[idx]["essay_id"]
        mask = [1] * len(ids)
        # print(len(ids),len(mask),len(label))
        return {
            "ids": ids,
            "mask": mask,
            "essay_id":essay_id,
            "targets": label,

        }



class Collate:
    def __init__(self, cfg):
        self.tokenizer = cfg.tokenizer
        self.cfg = cfg

    def __call__(self, batch):
        output = dict()
        output["ids"] = [sample["ids"] for sample in batch]
        output["mask"] = [sample["mask"] for sample in batch]
        output["essay_id"] = [sample["essay_id"] for sample in batch]
        output["targets"] = [sample["targets"] for sample in batch]
            
        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["ids"]])
    
        # add padding
        if self.tokenizer.padding_side == "right":
            output["ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["ids"]]
            output["mask"] = [s + (batch_max - len(s)) * [0] for s in output["mask"]]
        else:
            output["ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["ids"]]
            output["mask"] = [(batch_max - len(s)) * [0] + s for s in output["mask"]]
        
        # convert to tensors
        output["ids"] = torch.tensor(output["ids"], dtype=torch.long)
        output["mask"] = torch.tensor(output["mask"], dtype=torch.long)
        output["targets"] = torch.tensor(output["targets"], dtype=torch.float)
       
        
        # print(output['ids'].shape, output['mask'].shape, output['targets'].shape)
        
        return output

In [15]:
train_dataset = FeedbackDataset(training_samples_)
collate_fn = Collate(CFG)
train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,  collate_fn=collate_fn, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
for step, inputs in enumerate(train_loader):
  # print(step)
  break

In [16]:
class FeedBackModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()

        hidden_dropout_prob: float = 0.0
        self.model_config = AutoConfig.from_pretrained(model_name)

        self.model_config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "add_pooling_layer": False,
                "num_labels": CFG.num_labels,
                 "attention_probs_dropout_prob":0.0 
            }
        )

        self.model = AutoModel.from_pretrained(model_name, config=self.model_config)
        self.model.resize_token_embeddings(len(CFG.tokenizer))
        self.output = nn.Linear(self.model_config.hidden_size, CFG.num_labels)


    def forward(self, ids, mask, targets=None):
        transformer_out = self.model(input_ids=ids, attention_mask=mask)

        # Get CLS token for last 6 hidden layers
        h1 = transformer_out[1][-1][:,0,:].reshape((-1, 1, self.model_config.hidden_size)) #last 1 hidden state # CLS token (4,1,1024)
        h2 = transformer_out[1][-2][:,0,:].reshape((-1, 1, self.model_config.hidden_size)) #2nd last hidden state
        h3 = transformer_out[1][-3][:,0,:].reshape((-1, 1, self.model_config.hidden_size))
        h4 = transformer_out[1][-4][:,0,:].reshape((-1, 1, self.model_config.hidden_size))
        h5 = transformer_out[1][-5][:,0,:].reshape((-1, 1, self.model_config.hidden_size))
        h6 = transformer_out[1][-6][:,0,:].reshape((-1, 1, self.model_config.hidden_size))
        h7 = transformer_out[1][-7][:,0,:].reshape((-1, 1, self.model_config.hidden_size))
        h8 = transformer_out[1][-8][:,0,:].reshape((-1, 1, self.model_config.hidden_size))

        all_h = torch.cat([ h1, h2, h3, h4, h5, h6,h7,h8], 1)
        # Average CLS token for 6 hidden layers
        all_h_mean = torch.mean(all_h,1).reshape((-1, 1, self.model_config.hidden_size)) # torch.Size([bs, 1, hidden size])

        # Average CLS token for last hidden state + CLS token for 6 hidden layers 
        sequence_output = torch.mean(torch.cat([transformer_out.last_hidden_state[:,0,:].reshape((-1, 1, self.model_config.hidden_size)),
                                                all_h_mean],1),1)

        logits = self.output(sequence_output)
        return logits


In [17]:
# m = FeedBackModel(CFG.model)
# m(inputs['ids'], inputs['mask'])

In [18]:
class Loss_Fn(nn.Module):
    def __init__(self):
        super().__init__()
        self.lfn = nn.MSELoss()
    
    def forward(self, outputs, targets):
        loss = self.lfn(outputs, targets)
        return loss
        

In [19]:
from datasets.features.features import config

class AWP:
    def __init__(
        self,
        model,
        optimizer,
        adv_param = 'weight',
        adv_lr = 1,
        adv_eps = 0.2,
        start_step = 0,
        adv_step = 1,
        scaler = None
    ):
        self.model = model
        self.optimizer = optimizer
        self.adv_param = adv_param
        self.adv_lr = adv_lr
        self.adv_eps = adv_eps
        self.start_step = start_step
        self.adv_step = adv_step
        self.backup = {}
        self.backup_eps = {}
        self.scaler = scaler

    def attack_backward(self, batch, epoch):
        
        criterion = Loss_Fn()
        
        if (self.adv_lr == 0) or (epoch < self.start_step):
            return None

        self._save()
        for i in range(self.adv_step):
            self._attack_step() 
            with autocast(enabled = CFG.apex):
                input_ids = batch['ids'].to(CFG.device)
                attention_mask = batch['mask'].to(CFG.device)
                # token_type_ids = batch['token_type_ids'].to(cfg.device)
                labels = batch['targets'].to(CFG.device)
                tr_logits = self.model(input_ids, attention_mask,  labels)
                
                loss = criterion(tr_logits, labels)
                adv_loss = loss
                if CFG.gradient_accumulation_steps > 1:
                    adv_loss = loss / CFG.gradient_accumulation_steps
                    
#                 adv_loss = adv_loss.mean()
            self.optimizer.zero_grad()
            self.scaler.scale(adv_loss).backward()
            
        self._restore()

    def _attack_step(self):
        e = 1e-6
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                norm1 = torch.norm(param.grad)
                norm2 = torch.norm(param.data.detach())
                if norm1 != 0 and not torch.isnan(norm1):
                    r_at = self.adv_lr * param.grad / (norm1 + e) * (norm2 + e)
                    param.data.add_(r_at)
                    param.data = torch.min(
                        torch.max(param.data, self.backup_eps[name][0]), self.backup_eps[name][1]
                    )
                    
    def _save(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                if name not in self.backup:
                    self.backup[name] = param.data.clone()
                    grad_eps = self.adv_eps * param.abs().detach()
                    self.backup_eps[name] = (
                        self.backup[name] - grad_eps,
                        self.backup[name] + grad_eps,
                    )

    def _restore(self,):
        for name, param in self.model.named_parameters():
            if name in self.backup:
                param.data = self.backup[name]
        self.backup = {}
        self.backup_eps = {}

In [20]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

def get_score(outputs,targets): #outputs=preds, targets=groundtruth
    mcrmse = []
    for i in range(CFG.num_labels):
        mcrmse.append(
            metrics.mean_squared_error(
                targets[:, i],
                outputs[:, i],
                squared=False,
            ),
        )
    mcrmse = np.mean(mcrmse)
    return mcrmse # {"mcrmse": torch.tensor(mcrmse, device=device)}

In [21]:
def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device, valid_loader=None, valid_idx=None, best_score=np.inf):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    # best_score = np.inf


    if CFG.use_awp:
        # Initialize AWP
        log_awp=True
        awp = AWP(model, optimizer, adv_lr = CFG.adv_lr, adv_eps = CFG.adv_eps, start_step = CFG.start_awp_epoch, scaler = scaler)


    for step, inputs in enumerate(train_loader):
        model.train()
        gc.collect()
        # for k, v in inputs.items():
        #     inputs[k] = v.to(device)
        inputs['ids'] = inputs['ids'].to(device)
        inputs['mask'] = inputs['mask'].to(device)

        labels = inputs['targets'].to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs['ids'],inputs['mask'])
            # print(y_preds.shape, labels.shape, labels.view(-1, 1).shape)
            loss = criterion(y_preds, labels) #.view(-1, 1))
            
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        
        
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()

        if CFG.use_awp and best_score <= CFG.awp_score_check:
            if best_score <= CFG.awp_score_check and log_awp:
                LOGGER.info(' Start AWP '.center(50, '-'))
                log_awp=False
            if (step + 1) % CFG.gradient_accumulation_steps == 0:
                awp.attack_backward(inputs, epoch)
        

        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
        
        if CFG.val_strategy=="batch": 
            if step % CFG.val_steps == 0 or step == (len(train_loader)-1): # or if last step 
                  # Validate here
                  avg_val_loss, predictions, output_map = valid_fn(valid_loader, model, criterion, device, valid_idx)
                  
                  valid_df = df_folds[df_folds.text_id.isin(valid_idx)].copy()
                  for i,c in enumerate(true_cols):
                      valid_df.loc[:, f'pred_{c}'] = valid_df['text_id'].apply(lambda x: output_map[x][i])
                
                  valid_labels = valid_df[true_cols].values
                  valid_preds = valid_df[pred_cols].values

                  score = get_score(valid_preds,valid_labels) 
                  save_preds = valid_df[pred_cols].values

                  if  score < best_score:
                      best_score = score
                      LOGGER.info(f'Epoch {epoch+1} - Step {step}/{len(train_loader)} - Save Best Score: {best_score:.4f} Model')
                      torch.save({'model': model.state_dict(),
                                  'predictions': save_preds},
                                  OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")
                      

    return losses.avg, best_score



def valid_fn(valid_loader, model, criterion, device, valid_idx):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, inputs in enumerate(valid_loader):
        inputs['ids'] = inputs['ids'].to(device)
        inputs['mask'] = inputs['mask'].to(device)

        labels = inputs['targets'].to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs['ids'],inputs['mask'])
            loss = criterion(y_preds, labels) #.view(-1, 1))
            
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        
        losses.update(loss.item(), batch_size)
        
        preds.append(y_preds.detach().to('cpu').numpy())

        end = time.time()
        if step == (len(valid_loader)-1): #print only at the end 
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    
    preds = np.vstack(preds) 

    output_map = {}
    for x, y in zip(valid_idx, preds):
        output_map[x] = y.tolist()
    
    return losses.avg, preds, output_map


In [22]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
     
    seed_everything(seed=CFG.seed)

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_df = folds[folds["kfold"] != fold].reset_index(drop=True)#.head(500)
    valid_df = folds[folds["kfold"] == fold].reset_index(drop=True)

    pl_data_fold = pd.read_csv(f'/home/rashmi/Documents/kaggle/feedback3/input/fb3-psuedo-labels-1/exp_PL1_from_exp47_fold{fold}.csv')
    pl_data_fold = pl_data_fold.merge(all_pldata[['text_id','kfold']],on='text_id',how='left')
    pl_data_fold = pl_data_fold[train_df.columns]
    train_df = pd.concat([train_df,pl_data_fold[pl_data_fold.kfold==fold]])

    NUM_JOBS = os.cpu_count()
    # training_samples_ = prepare_training_data(train_df, CFG.tokenizer, num_jobs=NUM_JOBS, is_train=True)
    # valid_samples_ = prepare_training_data(valid_df, CFG.tokenizer, num_jobs=NUM_JOBS, is_train=True)

    # training_samples, valid_samples = training_samples_, valid_samples_

    # valid_idx = []
    # for x in valid_samples:
    #     valid_idx.append(x['essay_id'])  
    
    # print(len(valid_idx), valid_df.shape)

    # train_dataset = FeedbackDataset(training_samples)
    # valid_dataset = FeedbackDataset(valid_samples)

    # collate_fn = Collate(CFG)

    # train_loader = DataLoader(train_dataset,
    #                           batch_size=CFG.batch_size,
    #                           shuffle=True,  collate_fn=collate_fn, 
    #                           num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    # valid_loader = DataLoader(valid_dataset,
    #                           batch_size=CFG.batch_size,
    #                           shuffle=False, collate_fn=collate_fn, 
    #                           num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    # ====================================================
    # model & optimizer
    # ====================================================
    model = FeedBackModel(CFG.model) 
    torch.save(model.model_config, OUTPUT_DIR+'config.pth')
    model.to(device)

    if CFG.wandb: # this logs all gradients
        wandb.watch(model, log='all')
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
              'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
              'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
              'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    
    if CFG.gpu_optimize_config_adam:
        optimizer = bnb.optim.AdamW(optimizer_parameters, lr=CFG.encoder_lr, optim_bits=8)
    else:
        optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_df) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    # criterion = nn.BCEWithLogitsLoss(reduction="mean")
    # criterion = nn.CrossEntropyLoss()
    criterion = Loss_Fn() #MSECorrLoss()
    
    best_score = np.inf

    for epoch in range(CFG.epochs):
        if epoch==0:
            CFG.max_len = 768
        if epoch==1:
            CFG.max_len = 512
        if epoch==2:
            CFG.max_len = 512 
        else:
            CFG.max_len = 470
            
        training_samples_ = prepare_training_data(train_df, CFG.tokenizer, num_jobs=NUM_JOBS, is_train=True)
        valid_samples_ = prepare_training_data(valid_df, CFG.tokenizer, num_jobs=NUM_JOBS, is_train=True)

        training_samples, valid_samples = training_samples_, valid_samples_

        valid_idx = []
        for x in valid_samples:
            valid_idx.append(x['essay_id'])  

        print(len(valid_idx), valid_df.shape)

        train_dataset = FeedbackDataset(training_samples)
        valid_dataset = FeedbackDataset(valid_samples)

        collate_fn = Collate(CFG)

        train_loader = DataLoader(train_dataset,
                                  batch_size=CFG.batch_size,
                                  shuffle=True,  collate_fn=collate_fn, 
                                  num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
        valid_loader = DataLoader(valid_dataset,
                                  batch_size=CFG.batch_size,
                                  shuffle=False, collate_fn=collate_fn, 
                                  num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
        
        start_time = time.time()

        # train and validate
        avg_loss, best_score = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device, valid_loader, valid_idx, best_score)


    predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    
    valid_df['pred_cohesion'] = predictions[:,0]
    valid_df['pred_syntax'] = predictions[:,1]
    valid_df['pred_vocabulary'] = predictions[:,2]
  
    valid_df['pred_phraseology'] = predictions[:,3]
    valid_df['pred_grammar'] = predictions[:,4]
    valid_df['pred_conventions'] = predictions[:,5]
    

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_df

In [23]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = oof_df[true_cols].values
        preds = oof_df[pred_cols].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(df_folds, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                _oof_df.to_csv(OUTPUT_DIR+f'oof_df{fold}.csv',index=False)
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        oof_df.to_csv(OUTPUT_DIR+'oof_df.csv',index=False)
        
    if CFG.wandb:
        wandb.finish()

========== fold: 0 training ==========
Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

782 (782, 9)


Epoch: [1][0/1539] Elapsed 0m 1s (remain 43m 4s) Loss: 8.8496(8.8496) Grad: inf  LR: 0.00000010  


Epoch 1 - Step 0/1539 - Save Best Score: 2.9045 Model


EVAL: [195/196] Elapsed 0m 23s (remain 0m 0s) Loss: 10.1507(8.5389) 
Epoch: [1][100/1539] Elapsed 1m 5s (remain 15m 26s) Loss: 0.3086(7.9381) Grad: 57168.4961  LR: 0.00001000  
Epoch: [1][200/1539] Elapsed 1m 42s (remain 11m 24s) Loss: 0.1747(4.1039) Grad: 18277.2246  LR: 0.00000999  


...

EVAL: [195/196] Elapsed 0m 21s (remain 0m 0s) Loss: 0.1449(0.1965) 
Epoch: [4][800/1539] Elapsed 9m 34s (remain 8m 49s) Loss: 0.0042(0.0745) Grad: 128234.0312  LR: 0.00000036  
Epoch: [4][900/1539] Elapsed 10m 35s (remain 7m 30s) Loss: 0.2112(0.0747) Grad: 414776.8750  LR: 0.00000027  
Epoch: [4][1000/1539] Elapsed 11m 36s (remain 6m 14s) Loss: 0.0832(0.0749) Grad: 123096.0781  LR: 0.00000019  


Epoch 4 - Step 1000/1539 - Save Best Score: 0.4427 Model


EVAL: [195/196] Elapsed 0m 21s (remain 0m 0s) Loss: 0.1443(0.1964) 
Epoch: [4][1100/1539] Elapsed 13m 1s (remain 5m 10s) Loss: 0.0543(0.0748) Grad: 102284.5312  LR: 0.00000013  
Epoch: [4][1200/1539] Elapsed 14m 2s (remain 3m 57s) Loss: 0.0021(0.0756) Grad: 35553.8945  LR: 0.00000008  
EVAL: [195/196] Elapsed 0m 22s (remain 0m 0s) Loss: 0.1415(0.1965) 
Epoch: [4][1300/1539] Elapsed 15m 25s (remain 2m 49s) Loss: 0.1578(0.0758) Grad: 165586.1875  LR: 0.00000004  
Epoch: [4][1400/1539] Elapsed 16m 26s (remain 1m 37s) Loss: 0.0181(0.0766) Grad: 32381.0156  LR: 0.00000001  
Epoch: [4][1500/1539] Elapsed 17m 26s (remain 0m 26s) Loss: 0.0340(0.0765) Grad: 47045.3398  LR: 0.00000000  
EVAL: [195/196] Elapsed 0m 21s (remain 0m 0s) Loss: 0.1412(0.1966) 
Epoch: [4][1538/1539] Elapsed 18m 10s (remain 0m 0s) Loss: 0.0953(0.0764) Grad: 58932.4844  LR: 0.00000000  
EVAL: [195/196] Elapsed 0m 22s (remain 0m 0s) Loss: 0.1412(0.1966) 


========== fold: 4 result ==========
Score: 0.4427
========== CV ==========
Score: 0.4480


[fold0] loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
[fold0] lr,▅██████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
[fold1] loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
[fold1] lr,▅██████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
[fold2] loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
[fold2] lr,▅██████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
[fold3] loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
[fold3] lr,▅██████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
[fold4] loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
[fold4] lr,▅██████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
[fold0] loss,0.06943


In [24]:
CFG.tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')

('/home/rashmi/Documents/kaggle/feedback3//src/models_exp117a_PL_9/tokenizer/tokenizer_config.json',
 '/home/rashmi/Documents/kaggle/feedback3//src/models_exp117a_PL_9/tokenizer/special_tokens_map.json',
 '/home/rashmi/Documents/kaggle/feedback3//src/models_exp117a_PL_9/tokenizer/spm.model',
 '/home/rashmi/Documents/kaggle/feedback3//src/models_exp117a_PL_9/tokenizer/added_tokens.json',
 '/home/rashmi/Documents/kaggle/feedback3//src/models_exp117a_PL_9/tokenizer/tokenizer.json')

In [25]:
!nvidia-smi

Thu Nov 17 03:55:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.56.06    Driver Version: 520.56.06    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  Off |
| 31%   47C    P2    72W / 450W |  10847MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
if colab:
    from google.colab import runtime
    runtime.unassign()